<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Generator functions and coroutines</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_130_functions/topic_370_e4_generator_functions</div>



## Generator functions

More complex cases can no longer be covered by generator expressions.

- Generator that generates all numbers (no upper limit)
- Generator that modifies an iterable (e.g. executes multiple times, takes a fixed
  number of elements)

For these cases there are generator functions.


By using the `yield` keyword a function can "return multiple times." A
function that uses `yield` in its body is called a generator function. Simply
calling a generator function will not evaluate the function body, instead it
will return a *generator* that can return multiple times:

In [ ]:
def integers(start=0):
    n = start
    while True:
        yield n
        n += 1

In [ ]:
gen = integers()
print(repr(gen))
print(repr(iter(gen)))

In [ ]:
gen = integers()

In [ ]:
next(gen)

In [ ]:
next(gen)

In [ ]:
for i in integers():
    if i > 3:
        break
    print(i, end=" ")


## Mini-Workshop

Write a generator function `one_based_range(n)` that simulates `range(n)` when
used with a single argument but iterates from 1 to `n` inclusive.

In [ ]:
def one_based_range(n):
    for i in range(1, n + 1):
        yield i

In [ ]:
assert list(one_based_range(4)) == [1, 2, 3, 4]


Write a generator function `inclusive_range()` that simulates the complete
functionality of `range()` (i.e., being able to be called with one, two or
three arguments) but includes its upper limit.

Ensure that your implementation satisfies the provided test cases.

In [ ]:
from typing import overload

In [ ]:
@overload
def inclusive_range(end):
    ...


@overload
def inclusive_range(begin, end):
    ...

In [ ]:
def inclusive_range(begin, end=None, step=1):
    if end is None:
        end = begin
        begin = 1
    for i in range(begin, end + 1, step):
        yield i

In [ ]:
inclusive_range(1)

In [ ]:
assert list(inclusive_range(3)) == [1, 2, 3]

In [ ]:
assert list(inclusive_range(2, 4)) == [2, 3, 4]

In [ ]:
assert list(inclusive_range(2, 2)) == [2]

In [ ]:
assert list(inclusive_range(2, 1)) == []

In [ ]:
assert list(inclusive_range(2, 6, 2)) == [2, 4, 6]


We can use generator functions to write functions that process iterators. For
example, the `take()` generator function takes a fixed number of values from
an iterator:

In [ ]:
def take(n, it):
    for i in range(n):
        yield next(it)

In [ ]:
list(take(3, integers()))


Note that `drop()` function,  that removes the first `n` elements from an
iterator is not a generator function but a regular function:

In [ ]:
def drop(n, it):
    for i in range(n):
        next(it)
    return it

In [ ]:
list(take(3, drop(2, integers())))


Using generator function we can also define more complex iteration operations:

In [ ]:
def repeat_n_times(n, it):
    for _ in range(n):
        for elt in it:
            yield elt

In [ ]:
for num in repeat_n_times(3, range(5)):
    print(num, end=" ")

In [ ]:
def repeat_n_times_2(n, it):
    for _ in range(n):
        yield from it

In [ ]:
for num in repeat_n_times_2(3, range(5)):
    print(num, end=" ")


## Mini-workshop "Generators"

Write a generator  `leaves(tree: Bintree)` that returns all leaves from a binary tree (i.e.,
all nodes that are not instances of `Bintree`.

In [ ]:

from dataclasses import dataclass
from typing import Optional

In [ ]:
@dataclass
class Bintree:
    value: object = None
    left: Optional["Bintree"] = None
    right: Optional["Bintree"] = None

In [ ]:
def leaves(tree: Bintree):
    if tree:
        yield from leaves(tree.left)
        yield tree.value
        yield from leaves(tree.right)

In [ ]:
tree1 = Bintree(1)
tree2 = Bintree(2, Bintree(1), Bintree(3))
tree3 = Bintree(
    1, Bintree(0), Bintree(3, Bintree(2), Bintree(5, Bintree(4), Bintree(6)))
)

In [ ]:
assert tuple(leaves(tree1)) == (1,)

In [ ]:
assert tuple(leaves(tree2)) == (1, 2, 3)

In [ ]:
assert tuple(leaves(tree3)) == tuple(range(7))


# Coroutines

`yield` can also be used to return a value to the site where it is invoked. In
this case we also use the term *coroutine* for the generator.

Coroutines are useful building blocks for features such as cooperative
multitasking or event-based programming.

To start a coroutine `c` we call the method `c.send(None)`. To send subsequent
values we use `c.send(value)`.

In [ ]:
def my_coroutine(n):
    for i in range(n):
        x = yield
        print(x)

In [ ]:
c = my_coroutine(3)
print(c)

In [ ]:
# Initial value must be `None`
c.send(None)

In [ ]:
c.send(10)

In [ ]:
c.send(20)

In [ ]:
# c.send(40)

In [ ]:
def your_coroutine(n):
    for i in range(1, n + 1):
        x = yield i
        print("your_coroutine:", x)

In [ ]:
c = your_coroutine(3)

In [ ]:
_x = c.send(None)
print("top level:", _x)

In [ ]:
try:
    while True:
        _x = c.send(_x * 10)
        print("top level:", _x)
except StopIteration:
    print("Done.")

In [ ]:
def counter(max_value):
    i = 0
    while i < max_value:
        val = yield i
        if val is not None:
            i = val
        else:
            i += 1

In [ ]:
it = counter(100)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
it.send(98)

In [ ]:
next(it)

In [ ]:
# next(it)